Data augmentation

In [1]:
import cv2
import numpy as np
import os

# Function to apply data augmentation on a video frame
def augment_frame(frame):
    augmented_frames = []

    # 1. Horizontal Flip
    flipped = cv2.flip(frame, 1)
    augmented_frames.append(flipped)

    # 2. Horizontal Shift
    tx = 50  # Horizontal shift in pixels
    rows, cols = frame.shape[:2]
    M_shift = np.float32([[1, 0, tx], [0, 1, 0]])
    shifted = cv2.warpAffine(frame, M_shift, (cols, rows))
    augmented_frames.append(shifted)

    # 3. Scaling
    scale = 1.2  # Scale factor (>1: zoom out, <1: zoom in)
    scaled = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    scaled = cv2.resize(scaled, (cols, rows))
    augmented_frames.append(scaled)

    # 4. Brightness Adjustment
    brightness_factor = 50
    brightened = cv2.convertScaleAbs(frame, alpha=1, beta=brightness_factor)
    augmented_frames.append(brightened)

    # 5. Sharpening
    sharpening_kernel = np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])
    sharpened = cv2.filter2D(frame, -1, sharpening_kernel)
    augmented_frames.append(sharpened)

    # 6. Gaussian Blur
    blurred = cv2.GaussianBlur(frame, (7, 7), 1.5)
    augmented_frames.append(blurred)

    return augmented_frames


# Function to process and augment video
def augment_video(input_video_path, output_dir):
    # Create output directory if not exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Detect file extension
    file_name, file_ext = os.path.splitext(os.path.basename(input_video_path))

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {input_video_path}")
        return

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Compatible with most formats

    # Output file writers for each augmentation (retain the same file extension)
    writers = {
        "flipped": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_flipped{file_ext}"), fourcc, fps, (width, height)),
        "shifted": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_shifted{file_ext}"), fourcc, fps, (width, height)),
        "scaled": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_scaled{file_ext}"), fourcc, fps, (width, height)),
        "brightened": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_brightened{file_ext}"), fourcc, fps, (width, height)),
        "sharpened": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_sharpened{file_ext}"), fourcc, fps, (width, height)),
        "blurred": cv2.VideoWriter(os.path.join(output_dir, f"{file_name}_blurred{file_ext}"), fourcc, fps, (width, height)),
    }

    # Process video frame by frame
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        augmented_frames = augment_frame(frame)

        # Save each augmented frame
        for (key, writer), aug_frame in zip(writers.items(), augmented_frames):
            writer.write(cv2.resize(aug_frame, (width, height)))

        frame_count += 1
        if frame_count % 50 == 0:
            print(f"Processed {frame_count} frames in {file_name}...")

    # Release resources
    cap.release()
    for writer in writers.values():
        writer.release()

    print(f"Augmentation complete for video: {input_video_path}")


# Function to process all videos in a folder
def process_all_videos(input_folder, output_folder):
    # Create output folder if not exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all video files in the folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov')):  # Add supported video formats here
            video_path = os.path.join(input_folder, filename)
            print(f"Processing video: {video_path}")
            augment_video(video_path, output_folder)

    print("All videos processed successfully!")


# Main Execution
if __name__ == "__main__":
    input_folder = "../model_input/train_clips"  # Folder with input videos
    output_folder = "../model_input/augmented_train_clips"  # Folder for augmented videos

    process_all_videos(input_folder, output_folder)

Processing video: ../model_input/train_clips\april_0.MOV
Processed 50 frames in april_0...
Augmentation complete for video: ../model_input/train_clips\april_0.MOV
Processing video: ../model_input/train_clips\april_1.MOV
Augmentation complete for video: ../model_input/train_clips\april_1.MOV
Processing video: ../model_input/train_clips\april_11.MOV
Processed 50 frames in april_11...
Augmentation complete for video: ../model_input/train_clips\april_11.MOV
Processing video: ../model_input/train_clips\april_12.MOV
Processed 50 frames in april_12...
Augmentation complete for video: ../model_input/train_clips\april_12.MOV
Processing video: ../model_input/train_clips\april_13.MOV
Processed 50 frames in april_13...
Augmentation complete for video: ../model_input/train_clips\april_13.MOV
Processing video: ../model_input/train_clips\april_14.MOV
Processed 50 frames in april_14...
Augmentation complete for video: ../model_input/train_clips\april_14.MOV
Processing video: ../model_input/train_clips